In [1]:
#libraries
import pandas as pd
import numpy as np
import operator
from functools import reduce
import re
pd.set_option('display.max_rows', 100)

In [2]:
amazon_df = pd.read_csv("AmazonBooksData.csv")  # load data

In [3]:
amazon_df.head()  # display data

,priceFromates,prices,rating,title
0,\n \n \n \n Kindle...,"$,0,.,00,$,11,.,99,$,0,.,00",4.3 out of 5 stars,Last Day
1,\n \n \n \n Hardco...,"$,28,.,95,$,19,.,99",4.7 out of 5 stars,The Last Winter of the Weimar Republic: The Ri...
2,\n \n \n \n Paperb...,"$,10,.,69,$,3,.,99,$,24,.,99",4.6 out of 5 stars,"You Can Draw in 30 Days: The Fun, Easy Way to ..."
3,\n \n \n \n Hardco...,"$,26,.,33",4.8 out of 5 stars,The Last Kids on Earth: The Monster Box (books...
4,\n \n \n \n Hardco...,"$,16,.,22,$,0,.,00",4.6 out of 5 stars,The Whole30: The 30-Day Guide to Total Health ...


In [4]:
# clean prices column and adjust them in following formate dollar.cents
def cleanPrice(prices):
    if pd.isna(prices):
        prices = "0"    # set Nan values to 0
    splitPrices  = prices.split(",")
    splitPrices = [x for x in splitPrices if x != "$"]    # remove "$" from prices
    newPrices = []
    for i in range(0, len(splitPrices),3):
        j = i+3
        newPrices.append(["".join(splitPrices[i:j])])   # adjust prices in following formate dollar.cents
    return newPrices
    

In [5]:
amazon_df["prices"] = amazon_df["prices"].apply(cleanPrice)
amazon_df["prices"][:5]

0     [[0.00], [11.99], [0.00]]
1            [[28.95], [19.99]]
2    [[10.69], [3.99], [24.99]]
3                     [[26.33]]
4             [[16.22], [0.00]]
Name: prices, dtype: object

In [6]:
#clean PriceFromates
def cleanPriceFormate(priceFromates):
    if pd.isna(priceFromates):
        priceType = ""          # set Nan to ""
    priceFromates = priceFromates.replace('\n', '')  # remove new lines
    priceFromates = re.sub(' +', ' ', priceFromates)   # replace mutiple spaces with single space
    splitPriceFormate  = priceFromates.split(",")      # splite on comma
    return splitPriceFormate

In [7]:
amazon_df["priceFromates"] = amazon_df["priceFromates"].apply(cleanPriceFormate)
amazon_df["priceFromates"][:5]

0    [ Kindle ,  Paperback ,  Audible Audiobook ]
1                         [ Hardcover ,  Kindle ]
2         [ Paperback ,  Kindle ,  Spiral-bound ]
3                                   [ Hardcover ]
4            [ Hardcover ,  Kindle ,  Paperback ]
Name: priceFromates, dtype: object

In [8]:
#clean Ratings
def cleanRating(rating):
    if pd.isna(rating):
        rating = "0"    # replace Nan with 0
    rating = rating.split()[0]   # extract rating 
    return rating

In [9]:
amazon_df["rating"] = amazon_df["rating"].apply(cleanRating)
amazon_df["rating"][:5]

0    4.3
1    4.7
2    4.6
3    4.8
4    4.6
Name: rating, dtype: object

In [10]:
# clean extra price formate tags
def cleanPriceAndPriceFormate(price, priceFormate):
    noOfPrices = len(price)
    priceFormate = priceFormate[0:noOfPrices]
    return priceFormate

In [11]:
amazon_df["priceFromates"] =  amazon_df[['prices','priceFromates']].apply(lambda x: cleanPriceAndPriceFormate(*x), axis=1)

In [12]:
amazon_df = amazon_df[amazon_df['priceFromates'].map(lambda d: d != [""])] # drop rown with price fromate = 0

In [13]:
# get distint values of Price Formate to create new Columns
distintPriceFromates = list(set(reduce(operator.concat, amazon_df["priceFromates"])))
distintPriceFromates

[' Loose Leaf ',
 ' Imitation Leather ',
 ' Paperback ',
 ' Audible Audiobook ',
 ' Novelty Book ',
 ' Cards ',
 ' Prime Video ',
 ' Calendar ',
 ' eTextbook ',
 ' Map ',
 ' Flexibound ',
 ' Hardcover ',
 ' Spiral-bound ',
 ' Product Bundle ',
 ' Sheet music ',
 ' Mass Market Paperback ',
 ' Printed Access Code ',
 ' Plastic Comb ',
 ' Board book ',
 ' Kindle ',
 ' Audio CD ',
 ' MP3 CD ',
 ' Kindle & comiXology ',
 ' Diary ',
 ' Kindle Edition ']

In [14]:
#Create new columns for each Price Formate
for formate in distintPriceFromates:
    amazon_df[str(formate)] = ""

In [15]:
#Fill the price under related Price formate and None in rest
def fillPricesFromates(price, priceTypes , formateName ):
    
    for i, ptype in enumerate(priceTypes):
        if ptype == formateName:
            return price[i]
        

In [16]:
# Call fillPricesFromates for each distinct Price Formate for all rows
for formate in distintPriceFromates:
    amazon_df[str(formate)] = amazon_df[['prices','priceFromates']].apply(lambda x: fillPricesFromates(*x, formate), axis=1)

In [17]:
amazon_df # final data frame

,priceFromates,prices,rating,title,Loose Leaf,Imitation Leather,Paperback,Audible Audiobook,Novelty Book,Cards,...,Mass Market Paperback,Printed Access Code,Plastic Comb,Board book,Kindle,Audio CD,MP3 CD,Kindle & comiXology,Diary,Kindle Edition
0,"[ Kindle , Paperback , Audible Audiobook ]","[[0.00], [11.99], [0.00]]",4.3,Last Day,None,None,[11.99],[0.00],None,None,...,None,None,None,None,[0.00],None,None,None,None,None
1,"[ Hardcover , Kindle ]","[[28.95], [19.99]]",4.7,The Last Winter of the Weimar Republic: The Ri...,None,None,None,None,None,None,...,None,None,None,None,[19.99],None,None,None,None,None
2,"[ Paperback , Kindle , Spiral-bound ]","[[10.69], [3.99], [24.99]]",4.6,"You Can Draw in 30 Days: The Fun, Easy Way to ...",None,None,[10.69],None,None,None,...,None,None,None,None,[3.99],None,None,None,None,None
3,[ Hardcover ],[[26.33]],4.8,The Last Kids on Earth: The Monster Box (books...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"[ Hardcover , Kindle ]","[[16.22], [0.00]]",4.6,The Whole30: The 30-Day Guide to Total Health ...,None,None,None,None,None,None,...,None,None,None,None,[0.00],None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,[ Hardcover ],[[14.99]],4.5,"Waiting to Derail: Ryan Adams and Whiskeytown,...",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
302,"[ Kindle , Paperback , Audible Audiobook ]","[[9.99], [12.76], [0.00]]",4.3,The Edge of Lost,None,None,[12.76],[0.00],None,None,...,None,None,None,None,[9.99],None,None,None,None,None
303,"[ Paperback , Kindle ]","[[15.99], [9.99]]",4.7,Slow Cooked Paleo: 75 Real Food Recipes for Ef...,None,None,[15.99],None,None,None,...,None,None,None,None,[9.99],None,None,None,None,None
304,[ Board book ],[[21.92]],4.7,"World of Eric Carle, Animal Tales Sound Storyb...",None,None,None,None,None,None,...,None,None,None,[21.92],None,None,None,None,None,None
